<a href="https://colab.research.google.com/github/victornunesvidal/pucrj-mvp-machine-learning/blob/main/MVP_Modelo_Propensao_Afinidade_REV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O modelo a seguir é um modelo de classificação que trás um ranking de priorização dos produtos (codigo_produto) mais propensos a serem comprados por cada codigo_cliente considerando todas as variáveis disponíveis.

O dataset é um CSV com os seguintes campos separados por ";":

- codigo_cliente
- codigo_produto
- data_venda
- unidades_vendidas
- venda_bruta = valor da venda antes dos descontos
- net_sales = valor da venda após os descontos
- volume_kg = volume em kilogramas de cada venda
- sistema_de_vendas = sistema pelo qual a venda foi feita (as vendas digitais são indicadas por "02.BEES")
- tipologia = tipo do cliente (MERCEARIA, BAR, PADARIA, entre outros)
- nivel_cliente = nivel do cliente baseado no ticket médio, classificado entre AAA (ticket médio de 15 mil/mês), AA (7 mil por mês), A, B, C, D, E e X.

Os campos venda_bruta, net_sales e volume_kg foram retirados do modelo para evitar redundância com o campo unidades_vendidas, já que todos esses campos descrevem volume de vendas, só trocando a unidade de medida.

In [74]:
# 1. Carregar os dados

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

import requests
from io import StringIO

# Link do arquivo no Google Drive (modificado para download direto)
url = "https://drive.google.com/uc?export=download&id=1lh6LrxlSQ72aoZ1sJavKSZq-xXSw_ZZg"

# Baixar o conteúdo do arquivo
response = requests.get(url)
if response.status_code == 200:
    # Corrigindo possíveis problemas de delimitador
    conteudo_corrigido = response.content.decode('utf-8').replace("\t", ";")

    # Lendo o conteúdo como DataFrame
    data = pd.read_csv(StringIO(conteudo_corrigido), sep=';', decimal=',')

    # Exibindo as primeiras linhas
    print("Primeiras linhas do DataFrame:")
    print(data.head())

    # Exibir informações do DataFrame
    print("\nInformações do DataFrame:")
    print(data.info())
else:
    print(f"Erro ao baixar o arquivo: {response.status_code}")

Primeiras linhas do DataFrame:
   codigo_cliente codigo_produto               data_venda sistema_de_vendas  \
0       100164574              _  2023-10-31 00:00:00,000            01.MC1   
1       100164574              _  2023-11-30 00:00:00,000            01.MC1   
2       100164574              _  2024-01-31 00:00:00,000            01.MC1   
3       100164574              _  2024-02-29 00:00:00,000            01.MC1   
4       100164574              _  2024-03-28 00:00:00,000            01.MC1   

              tipologia nivel_cliente  unidades_vendidas  venda_bruta  \
0  AS 05 A 09 CHECK-OUT           AAA                  0          0.0   
1  AS 05 A 09 CHECK-OUT           AAA                  0          0.0   
2  AS 05 A 09 CHECK-OUT           AAA                  0          0.0   
3  AS 05 A 09 CHECK-OUT           AAA                  0          0.0   
4  AS 05 A 09 CHECK-OUT           AAA                  0          0.0   

   net_sales  volume_kg  
0        0.0        0.0  
1  

In [75]:
# 2. Preparação dos dados
# Criar a variável alvo: Produto mais propenso a ser comprado
# Supomos que cada cliente tende a comprar mais frequentemente o produto que comprou anteriormente
# Transformamos os dados para obter a última compra como referência para treinamento
data['data_venda'] = pd.to_datetime(data['data_venda'])
data.sort_values(['codigo_cliente', 'data_venda'], ascending=True, inplace=True)
data['target'] = data.groupby('codigo_cliente')['codigo_produto'].shift(-1)
data = data.dropna(subset=['target'])



<ipython-input-75-280c8ba88931>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['data_venda'] = pd.to_datetime(data['data_venda'])


In [76]:
# 3. Seleção de atributos e preparação

# Variáveis categóricas a serem codificadas
data['sistema_de_vendas'] = LabelEncoder().fit_transform(data['sistema_de_vendas'])
data['tipologia'] = LabelEncoder().fit_transform(data['tipologia'])
data['nivel_cliente'] = LabelEncoder().fit_transform(data['nivel_cliente'])
data['codigo_produto'] = LabelEncoder().fit_transform(data['codigo_produto'])

# Atributos e alvo
features = ['codigo_produto', 'unidades_vendidas', 'sistema_de_vendas', 'tipologia', 'nivel_cliente']
target = 'target'

X = data[features]
y = data[target]

# Excluir produtos com amostras insuficientes
min_samples = 5
product_counts = y.value_counts()
valid_products = product_counts[product_counts >= min_samples].index

# Filtrar o dataset
X = X[y.isin(valid_products)]
y = y[y.isin(valid_products)]

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

<ipython-input-76-8c84a512a555>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sistema_de_vendas'] = LabelEncoder().fit_transform(data['sistema_de_vendas'])
<ipython-input-76-8c84a512a555>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tipologia'] = LabelEncoder().fit_transform(data['tipologia'])
<ipython-input-76-8c84a512a555>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [77]:
# 4. Pipeline de processamento e modelagem
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['unidades_vendidas']),
        ('cat', 'passthrough', ['codigo_produto', 'sistema_de_vendas', 'tipologia', 'nivel_cliente'])
    ])

# Modelos para testar
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42),
    'DecisionTree': DecisionTreeClassifier(random_state=42)
}

# Hiperparâmetros para otimização
param_grids = {
    'RandomForest': {
        'model__n_estimators': [50, 100, 400],
        'model__max_depth': [5, 10, None]
    },
    'LogisticRegression': {
        'model__C': [0.01, 0.1, 1, 10, 100]
    },
    'DecisionTree': {
        'model__max_depth': [5, 10, 20, None]
    }
}

# Ajustar validação cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Pipeline e GridSearch
best_models = {}
accuracies = {}
for model_name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_models[model_name] = grid_search.best_estimator_
    y_pred = grid_search.best_estimator_.predict(X_test)
    accuracies[model_name] = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least popul

In [78]:
# Exibir os melhores modelos e suas acurácias
for model_name, best_model in best_models.items():
    print(f"Melhor modelo para {model_name}: {best_model}")
    print(f"Acurácia para {model_name}: {accuracies[model_name]:.4f}")

Melhor modelo para RandomForest: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['unidades_vendidas']),
                                                 ('cat', 'passthrough',
                                                  ['codigo_produto',
                                                   'sistema_de_vendas',
                                                   'tipologia',
                                                   'nivel_cliente'])])),
                ('model',
                 RandomForestClassifier(max_depth=10, n_estimators=50,
                                        random_state=42))])
Acurácia para RandomForest: 0.4130
Melhor modelo para LogisticRegression: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['unidades_vendidas']),
                      

In [79]:
# Gerar ranking de produtos por cliente
X_test['codigo_cliente'] = data.loc[X_test.index, 'codigo_cliente']  # Restaurar códigos de cliente
predictions = pd.DataFrame({
    'codigo_cliente': X_test['codigo_cliente'],
    'codigo_produto_predito': best_models['RandomForest'].predict(X_test.drop(columns=['codigo_cliente']))
})

# Gerar ranking agrupado por cliente
ranking = predictions.groupby('codigo_cliente')['codigo_produto_predito'].apply(list).reset_index()
print(ranking.head())


   codigo_cliente                             codigo_produto_predito
0       100164574  [TORTILHA_PRA MIM, SALGADINHO DE MILHO 2_PRA G...
1       100186750  [PRODUTO7_PRA GALERA, BATATAS PREMIUM_PRA GENT...
2       100250696  [BATATAS_PRA GENTE, TORTILHA_PRA GALERA, TORTI...
3       100250697  [TORTILHA_PRA GALERA, SALGADINHO DE MILHO 1_PR...
4       100250802  [TORTILHA_PRA MIM, BATATAS_PRA GENTE, SALGADIN...


In [ ]:
# Análise de Resultados

O código realiza a previsão do próximo produto que um cliente comprará com base em dados históricos de vendas. Vamos analisar os passos e os resultados:


**1. Carregamento e Preparação dos Dados:**

* Os dados são carregados de um arquivo CSV no Google Drive. O código lida com possíveis problemas de delimitadores, substituindo tabulações por ponto e vírgula.
* A variável alvo 'target' é criada representando o código do produto comprado pelo cliente na próxima compra. A ordenação por cliente e data é crucial para essa definição.
* As colunas categóricas são codificadas usando `LabelEncoder`, transformando strings em números.
* Atributos irrelevantes são removidos e o dataset é dividido em treino e teste.

**2. Processamento e Modelagem:**

* Um `ColumnTransformer` é usado para pré-processar os dados: `StandardScaler` para a coluna numérica 'unidades_vendidas' e 'passthrough' para as categóricas.
* Três modelos de classificação são treinados e testados: RandomForestClassifier, LogisticRegression e DecisionTreeClassifier.
* Um `GridSearchCV` otimiza os hiperparâmetros de cada modelo utilizando validação cruzada estratificada (`StratifiedKFold`) para encontrar a melhor configuração.
* A acurácia de cada modelo no conjunto de teste é calculada.

**3. Resultados e Análise:**

* O código imprime o melhor modelo e a acurácia para cada algoritmo testado.  A acurácia é uma métrica importante, mas, dependendo do contexto e do desbalanceamento das classes, outras métricas como precisão, recall, F1-score e AUC podem ser mais relevantes.
* Um ranking dos produtos mais propensos a serem comprados por cada cliente é gerado usando as previsões do melhor modelo (RandomForest). A saída é um DataFrame com o código do cliente e uma lista dos produtos previstos.


**Pontos a serem considerados:**

Em resumo, o código fornece uma base sólida para previsão da próxima compra, mas exige ainda um refinamento e análise mais aprofundada para garantir a qualidade das predições.  Será necessário avaliar mais as métricas, realizar testes mais robustos, considerar outras variáveis e validar as hipóteses para melhorar a precisão e a confiabilidade do modelo.
